# LFT 03 — Dynamics & Time (N=4→5)

In Logic Field Theory (LFT), **time** is the directed application of the logical operator \(L\) that reduces inconsistency and increases order. This notebook formalizes time as a **Lyapunov descent** on total orders (permutations) and extends it to partial orders:

1. Define the **inversion count** \(h(\sigma)\) on \(S_N\).
2. Prove that **adjacent swaps** that resolve inversions **strictly decrease** \(h\).
3. Simulate the descent for **N=4** and **N=5** from random and worst-case starts.
4. Extend to **partial orders** with a potential \(\tilde H\), verified on a concrete DAG example (N=4).

## 1. Inversion count and the bubble-sort lemma

**Definition.** For a permutation \(\sigma\in S_N\), the **inversion count**
$$h(\sigma)=\#\{(i,j)\mid 1\le i<j\le N,\ \sigma(i) > \sigma(j)\}$$
is the Kendall–Tau distance to the identity.

**Lemma (Bubble-sort step).** If an update performs an **adjacent swap** on indices \(k,k{+}1\) **only when** \(\sigma(k) > \sigma(k{+}1)\), then \(h\) decreases by exactly 1.

**Proof.** Only the pair \((k,k{+}1)\) changes relative order; if it was an inversion, swapping removes it and introduces no new inversions. ∎

In [ ]:
import numpy as np

def inversion_count(perm):
    inv = 0
    for i in range(len(perm)):
        for j in range(i+1, len(perm)):
            if perm[i] > perm[j]:
                inv += 1
    return inv

def local_descent_step(perm):
    """Pick a random adjacent pair; swap if it resolves an inversion."""
    N = len(perm)
    k = np.random.randint(0, N-1)
    p = list(perm)
    if p[k] > p[k+1]:
        p[k], p[k+1] = p[k+1], p[k]
    return tuple(p)

def simulate_h_flow(N, trials=500, steps=80, start="random", seed=0):
    rng = np.random.default_rng(seed)
    H = np.zeros((trials, steps+1), dtype=float)
    for t in range(trials):
        if start == "random":
            p = list(range(N))
            rng.shuffle(p)
            p = tuple(p)
        elif start == "worst":
            p = tuple(range(N-1, -1, -1))
        else:
            raise ValueError("start must be 'random' or 'worst'")
        H[t,0] = inversion_count(p)
        for s in range(1, steps+1):
            # one local step per time unit
            p = local_descent_step(p)
            H[t,s] = inversion_count(p)
    return H


## 2. Simulations: monotone descent (N=4 and N=5)
We run many trials from both **random** starts and the **worst** start (reverse order), then plot the mean and interquartile bands to confirm monotone descent of \(h(t)\).

In [ ]:
import matplotlib.pyplot as plt, os
os.makedirs('./outputs', exist_ok=True)

def plot_h_summary(H, title, outpath):
    t = np.arange(H.shape[1])
    mean_h = H.mean(axis=0)
    q25 = np.quantile(H, 0.25, axis=0)
    q75 = np.quantile(H, 0.75, axis=0)
    plt.figure()
    plt.plot(t, mean_h, label='mean h(t)')
    plt.fill_between(t, q25, q75, alpha=0.3, label='25–75%')
    plt.xlabel('Step')
    plt.ylabel('Inversions h(t)')
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.savefig(outpath, dpi=150)
    plt.close()

for N in [4, 5]:
    H_rand = simulate_h_flow(N, trials=800, steps=100, start='random', seed=123)
    H_worst = simulate_h_flow(N, trials=200, steps=100, start='worst', seed=456)
    plot_h_summary(H_rand, f'N={N} local L-flow: random starts', f'./outputs/N{N}_h_flow_random.png')
    plot_h_summary(H_worst, f'N={N} local L-flow: worst starts', f'./outputs/N{N}_h_flow_worst.png')
print('Saved h(t) plots for N=4 and N=5 into ./outputs')

## 3. Partial orders: extending the potential to \(\tilde H\)

For a DAG (partial order) \(P\), define \(\mathcal{L}(P)\) = set of **linear extensions** (topological sorts). Fix a reference total order (identity). Define
$$\tilde H(P) = \min_{\sigma\in\mathcal{L}(P)} \mathrm{KT}(\sigma, \mathrm{id}),$$
the minimum Kendall–Tau distance to identity over all linear extensions. \(\tilde H\) equals \(h\) on total orders.

**Proposition (Monotonicity under refinement).** If an update adds a comparability \(u\prec v\) that is consistent (keeps acyclicity), then \(\mathcal{L}(P)\) can only shrink and \(\tilde H\) is **non-increasing**. If all extensions must now resolve an adjacent inversion, \(\tilde H\) strictly decreases.

In [ ]:
import networkx as nx
from itertools import permutations

def kt_distance(order):
    # Kendall–Tau distance to identity for a permutation (tuple)
    return inversion_count(order)

def linear_extensions_min_kt(dag_edges, N):
    G = nx.DiGraph()
    G.add_nodes_from(range(N))
    G.add_edges_from(dag_edges)
    assert nx.is_directed_acyclic_graph(G)
    best = None
    for topo in nx.all_topological_sorts(G):
        d = kt_distance(tuple(topo))
        if best is None or d < best:
            best = d
    return best

# Concrete N=4 example DAG and its refinement
N = 4
P_edges = [(0,2), (1,2)]   # 0<2 and 1<2; 3 free
# Compute \tilde H before refinement
H_before = linear_extensions_min_kt(P_edges, N)

# Add a consistent comparability (2,3)
P_refined = P_edges + [(2,3)]
H_after = linear_extensions_min_kt(P_refined, N)
print({'tildeH_before': H_before, '\\tildeH_after': H_after, 'non_increasing?': H_after <= H_before})

## 4. Takeaways
- On total orders, **time** corresponds to a monotone descent of **\(h\)** under local L-steps (adjacent swaps that remove inversions).
- On partial orders, the extended potential **\(\tilde H\)** is **non-increasing** under consistent refinements and strictly decreases when an adjacent inversion must be resolved across all linear extensions.
- For **N=5**, these properties continue to hold; the geometry lives in \(V\cong\mathbb{R}^4\) (rank 4), with **time as L-flow** rather than an extra simple root.